In [3]:
# importing lib

import numpy as np
import pandas as pd
import nltk
import re
import string

In [6]:
# Importing Dataframe
input_df = pd.read_excel('input.xlsx')
input_df.head()

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...


In [7]:
# importing stopwords

with  open('StopWords_Generic.txt') as f1:
    stopword_business = f1.read()

stopword = re.sub('\n', ' ', stopword_business).lower()
stopword

#importing punct

punct = string.punctuation

In [8]:
# Importing sentiment csv

senti  = pd.read_csv('senti.csv', usecols=['Word', 'Negative', 'Positive'] )
senti = senti.dropna()
senti.head(10)

,Word,Negative,Positive
0,AARDVARK,0,0
1,AARDVARKS,0,0
2,ABACI,0,0
3,ABACK,0,0
4,ABACUS,0,0
5,ABACUSES,0,0
6,ABAFT,0,0
7,ABALONE,0,0
8,ABALONES,0,0
9,ABANDON,2009,0


In [9]:
# In Tokenization we usually lower() the word so will be impossible to find US that's whay i am doing word tokenize.

def pronoun(str):
    count = 0
    
    pro = ['I','we','my', 'ours','us', 'i', 'We', 'My', 'Ours']
    for token in nltk.word_tokenize(str):
        if token in pro:
            count += 1
        if token == 'US':
            count -= 1

    return count


In [10]:
# Specific function for syllables

def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('ed'):
        count -= 1
    if word.endswith('es'):
        count -= 1
    
    return count


In [11]:
# Function return values if the word is neg(-1), pos(+1), neutral(0)

def score(word):
    word = word.upper()
    sent =  senti[senti['Word'].str.contains(word)]
    pos = sent['Positive']
    neg = sent['Negative']
    
    if pos.all():
        return 1
    elif neg.all():
        return -1
    else:
        return 0

In [12]:
# for sentence preprocessing of string

def clean_data(str, file_name):
    clean_text = ''
    total_sent = 0
    total_word = 0
    complex_word = 0
    pos = 0
    neg = 0
    char = 0

    for sent in nltk.sent_tokenize(str):
        total_sent += 1
        for word in nltk.word_tokenize(sent.lower()):
            if word not in stopword:
                if word not in punct:
                    total_word+= 1

                    clean_text += ' '+word

                    if syllables(word)>2:
                        complex_word += 1

                    scor = score(word)               #pos and neg score   
                    if scor==1:
                        pos += 1
                    elif scor==-1:
                        neg +=1



                    for i in word:                   #To count total no of char
                        char += 1


    total_syllable = syllables(clean_text)

    pos
    neg
    polarity_score = (pos-neg)/((pos+neg)+0.000001)
    subjectivity_score = (pos+neg)/(total_word + 0.000001)

    avg_word_sent  = total_word/total_sent
    per_complex_word = (complex_word/total_word)*100
    fog_index = 0.4*(avg_word_sent + per_complex_word)

    avg_word_sent = total_word/total_sent

    complex_word
    total_word

    syllable_per_word = total_syllable/total_word
    per_pro = pronoun(str)
    avg_word_len = char/total_word

    data = {'Name':[file_name], 'Positive Score':[pos], 'Negative Score': [neg], 'Polarity Score': [polarity_score],
         'SUBJECTIVITY SCORE':[subjectivity_score], 'AVG SENTENCE LENGTH':[avg_word_sent], 
          'PERCENTAGE OF COMPLEX WORDS':[per_complex_word], 'FOG INDEX':[fog_index], 
          'AVG NUMBER OF WORDS PER SENTENCE':[avg_word_sent], 'COMPLEX WORD COUNT':[complex_word],
          'WORD COUNT': [total_word], 'SYLLABLE PER WORD':[syllable_per_word], 'PERSONAL PRONOUNS':[per_pro],
          'AVG WORD LENGTH':[avg_word_len]
         }
    
    return data

In [27]:
# Reading every file in the data folder and editing

output_df = pd.DataFrame()

for i in range(1,170):
    with open('data/{0}.txt'.format(i), 'r+',encoding="utf-8") as f:
        txt = f.read()
        
        
        output_df = output_df.append(clean_data(txt,input_df['URL'][i]), ignore_index=True)
        
    


In [14]:
output_df

,AVG NUMBER OF WORDS PER SENTENCE,AVG SENTENCE LENGTH,AVG WORD LENGTH,COMPLEX WORD COUNT,FOG INDEX,Name,Negative Score,PERCENTAGE OF COMPLEX WORDS,PERSONAL PRONOUNS,Polarity Score,Positive Score,SUBJECTIVITY SCORE,SYLLABLE PER WORD,WORD COUNT
0,[17.333333333333332],[17.333333333333332],[6.543269230769231],[117],[18.183333333333334],[https://insights.blackcoffer.com/how-does-ai-...,[0],[28.125],[4],[0.9999999687500011],[32],[0.07692307673816569],[2.2163461538461537],[416]
1,[13.464285714285714],[13.464285714285714],[6.915119363395226],[136],[19.815422508525955],[https://insights.blackcoffer.com/ai-and-its-i...,[4],[36.07427055702917],[2],[0.5555555246913597],[14],[0.047745357963540166],[2.413793103448276],[377]
2,[13.736842105263158],[13.736842105263158],[7.07183908045977],[412],[21.280177455132083],[https://insights.blackcoffer.com/how-do-deep-...,[12],[39.46360153256705],[13],[0.7913043409451797],[103],[0.11015325659947005],[2.4262452107279695],[1044]
3,[17.266666666666666],[17.266666666666666],[6.953667953667954],[94],[21.424041184041187],[https://insights.blackcoffer.com/how-artifici...,[1],[36.293436293436294],[1],[0.6666665555555741],[5],[0.023166023076579063],[2.4324324324324325],[259]
4,[9.894736842105264],[9.894736842105264],[7.058510638297872],[148],[19.70257558790594],[https://insights.blackcoffer.com/how-are-gene...,[6],[39.361702127659576],[21],[0.4999999791666675],[18],[0.06382978706428248],[2.484042553191489],[376]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,[11.923076923076923],[11.923076923076923],[6.776344086021505],[175],[19.822994210090986],[https://insights.blackcoffer.com/role-big-dat...,[1],[37.634408602150536],[0],[0.9428571159183682],[34],[0.07526881704243266],[2.339784946236559],[465]
165,[11.859154929577464],[11.859154929577464],[6.8123515439429925],[288],[18.42537218560771],[https://insights.blackcoffer.com/sales-foreca...,[18],[34.204275534441805],[15],[0.34545453917355384],[37],[0.0653206650055574],[2.4121140142517814],[842]
166,[15.964285714285714],[15.964285714285714],[7.217002237136465],[182],[22.672067753275808],[https://insights.blackcoffer.com/detect-data-...,[3],[40.7158836689038],[0],[0.8181817933884306],[30],[0.07382550319054697],[2.519015659955257],[447]
167,[11.245283018867925],[11.245283018867925],[7.189597315436242],[258],[21.813549449157907],[https://insights.blackcoffer.com/data-exfiltr...,[27],[43.288590604026844],[6],[0.4065934021253472],[64],[0.1526845635022071],[2.563758389261745],[596]


In [25]:
# Reordering as requried

new_order = ['Name', 'Positive Score', 'Negative Score', 'Polarity Score',
         'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 
          'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 
          'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
          'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
          'AVG WORD LENGTH']

output_df = output_df.filter(new_order)
output_df

,Name,Positive Score,Negative Score,Polarity Score,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,[https://insights.blackcoffer.com/how-does-ai-...,[32],[0],[0.9999999687500011],[0.07692307673816569],[17.333333333333332],[28.125],[18.183333333333334],[17.333333333333332],[117],[416],[2.2163461538461537],[4],[6.543269230769231]
1,[https://insights.blackcoffer.com/ai-and-its-i...,[14],[4],[0.5555555246913597],[0.047745357963540166],[13.464285714285714],[36.07427055702917],[19.815422508525955],[13.464285714285714],[136],[377],[2.413793103448276],[2],[6.915119363395226]
2,[https://insights.blackcoffer.com/how-do-deep-...,[103],[12],[0.7913043409451797],[0.11015325659947005],[13.736842105263158],[39.46360153256705],[21.280177455132083],[13.736842105263158],[412],[1044],[2.4262452107279695],[13],[7.07183908045977]
3,[https://insights.blackcoffer.com/how-artifici...,[5],[1],[0.6666665555555741],[0.023166023076579063],[17.266666666666666],[36.293436293436294],[21.424041184041187],[17.266666666666666],[94],[259],[2.4324324324324325],[1],[6.953667953667954]
4,[https://insights.blackcoffer.com/how-are-gene...,[18],[6],[0.4999999791666675],[0.06382978706428248],[9.894736842105264],[39.361702127659576],[19.70257558790594],[9.894736842105264],[148],[376],[2.484042553191489],[21],[7.058510638297872]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,[https://insights.blackcoffer.com/role-big-dat...,[34],[1],[0.9428571159183682],[0.07526881704243266],[11.923076923076923],[37.634408602150536],[19.822994210090986],[11.923076923076923],[175],[465],[2.339784946236559],[0],[6.776344086021505]
165,[https://insights.blackcoffer.com/sales-foreca...,[37],[18],[0.34545453917355384],[0.0653206650055574],[11.859154929577464],[34.204275534441805],[18.42537218560771],[11.859154929577464],[288],[842],[2.4121140142517814],[15],[6.8123515439429925]
166,[https://insights.blackcoffer.com/detect-data-...,[30],[3],[0.8181817933884306],[0.07382550319054697],[15.964285714285714],[40.7158836689038],[22.672067753275808],[15.964285714285714],[182],[447],[2.519015659955257],[0],[7.217002237136465]
167,[https://insights.blackcoffer.com/data-exfiltr...,[64],[27],[0.4065934021253472],[0.1526845635022071],[11.245283018867925],[43.288590604026844],[21.813549449157907],[11.245283018867925],[258],[596],[2.563758389261745],[6],[7.189597315436242]


In [26]:
output_df.to_excel('output_df.xlsx')